In [1]:
# !pip install langchain
# !pip install langchain-openai

# Introduction to LangChain 

Working with LLMs involves in one way or another working with a specific type of abstraction: "Prompts".

However, in the practical context of day-to-day tasks we expect LLMs to perform, these prompts won't be some static and dead type of abstraction. Instead we'll work with dynamic prompts re-usable prompts.

# Lanchain

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) is a framework that allows you to connect LLMs together by allowing you to work with modular components like prompt templates and chains giving you immense flexibility in creating tailored solutions powered by the capabilities of large language models.


Its main features are:
- **Components**: abstractions for working with LMs
- **Off-the-shelf chains**: assembly of components for accomplishing certain higher-level tasks

LangChain facilitates the creation of complex pipelines that leverage the connection of components like chains, prompt templates, output parsers and others to compose intricate pipelines that give you everything you need to solve a wide variety of tasks.

At the core of LangChain, we have the following elements:

- Models
- Prompts
- Output parsers

**Models**

Models are nothing more than abstractions over the LLM APIs like the ChatGPT API.​

In [2]:
# Uncomment and add your api key!!!

# import os

# # Set OPENAI API Key

# os.environ["OPENAI_API_KEY"] = "your openai key"

# OR (load from .env file)

from dotenv import load_dotenv
load_dotenv("./.env")

True

In [3]:
# !pip install langchain
# !pip install langchain-openai

In [4]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-3.5-turbo-0125")

output = chat_model.invoke("I am teaching a live-training\
    about LLMs!")

print(output.content)

That's great! LLMs, or Master of Laws programs, are a valuable opportunity for lawyers and legal professionals to specialize in a specific area of law or gain expertise in a foreign legal system. During your live training, you can cover topics such as the benefits of pursuing an LLM, different types of LLM programs available, admissions requirements, and career opportunities for LLM graduates. You can also provide tips for success in an LLM program and answer any questions participants may have. Good luck with your training!


In [5]:
from langchain.chat_models import ChatOllama

model = ChatOllama(model="mistral", format_chat=True)

model.invoke("Hi!")

AIMessage(content='Hi there! How can I help you today?')

You can predict outputs from both LLMs and ChatModels:

Basic components are:

- Models
- Prompt templates
- Output parsers

In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Show me 5 examples\
    of this concept: {concept}")

prompt.format(concept="animal")

'Human: Show me 5 examples    of this concept: animal'

In [7]:
chain = prompt | chat_model

output = chain.invoke({"concept": "animal"})
output

AIMessage(content='1. A cat is a common household animal that is known for its independence and agility.\n2. A dog is a loyal and friendly animal that is often kept as a pet for companionship and security.\n3. A lion is a majestic and powerful animal that is known as the king of the jungle.\n4. A dolphin is an intelligent and playful animal that lives in the ocean and is known for its acrobatic abilities.\n5. A giraffe is a tall and graceful animal with a long neck that allows it to reach high leaves in trees for food.')

In [8]:
from IPython.display import Markdown

Markdown(output.content)

1. A cat is a common household animal that is known for its independence and agility.
2. A dog is a loyal and friendly animal that is often kept as a pet for companionship and security.
3. A lion is a majestic and powerful animal that is known as the king of the jungle.
4. A dolphin is an intelligent and playful animal that lives in the ocean and is known for its acrobatic abilities.
5. A giraffe is a tall and graceful animal with a long neck that allows it to reach high leaves in trees for food.

You can also use the predict method over a string input:

In [9]:
text = "What would be a good name for a dog that loves to nap??"
chat_model.invoke(text)

AIMessage(content='Snuggles')

**Prompts**

The same works for prompts. Now, prompts are pieces of text we feed to LLMs, and LangChain allows you to work with prompt templates.

Prompt Templates are useful abstractions for reusing prompts and they are used to provide context for the specific task that the language model needs to complete. 

A simple example is a `PromptTemplate` that formats a string into a prompt:

In [10]:
from langchain_core.prompts  import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("What is a good dog name for a dog that loves to {activity}?")
prompt.format(activity="sleeping")
# Output: "What is a good dog name for a dog that loves to nap?"

'Human: What is a good dog name for a dog that loves to sleeping?'

**Output Parsers**

OutputParsers convert the raw output from an LLM into a format that can be used downstream. Here is an example of an OutputParser that converts a comma-separated list into a list:

In [11]:
chain.invoke({"concept": "Landscapes"})

AIMessage(content='1. A serene mountain landscape with snow-capped peaks, lush green forests, and a calm lake reflecting the clear blue sky.\n2. A vast desert landscape with rolling sand dunes, sparse vegetation, and a bright sun shining in a cloudless sky.\n3. A coastal landscape with rocky cliffs, crashing waves, and a sandy beach stretching into the distance.\n4. A peaceful countryside landscape with rolling hills, picturesque farms, and colorful fields of flowers.\n5. An urban landscape with towering skyscrapers, bustling city streets, and a mix of modern architecture and historical buildings.')

In [12]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [13]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
prompt = ChatPromptTemplate.from_template("""
Write 5 concepts that are fundamental to learn about {topic}.
                                          """)
chain = prompt | llm | output_parser
chain.invoke({"topic": "Artificial Neural Networks"})

'1. Neuron: Neurons are the basic building blocks of artificial neural networks, and they are modeled after the biological neurons in the human brain. Neurons take input signals, process them using weights and activation functions, and produce an output signal.\n\n2. Activation function: Activation functions are mathematical functions that determine the output of a neuron. They introduce non-linearity into the neural network, allowing it to learn complex patterns and relationships in the data.\n\n3. Backpropagation: Backpropagation is a key algorithm used to train artificial neural networks. It involves calculating the error between the predicted output of the network and the actual output, and then adjusting the weights of the neurons to minimize this error.\n\n4. Layers: Artificial neural networks are typically organized into layers, with each layer containing a set of neurons. The input layer receives the input data, the hidden layers process this data through weighted connections, 

This chain will take input variables, pass those to a prompt template to create a prompt, pass the prompt to an LLM, and then pass the output through an output parser.

Ok, so these are the basics of langchain. But how can we leverage these abstraction capabilities inside our LLM app application?

Now, to put everything together LangChain allows you to build something called "chains", which are components that connect prompts, llms and output parsers into a building block that allows you to create more interesting and complex functionality.

Let's look at the example below:

In [14]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("What is a good dog name for a dog that loves to {activity}?")

chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=prompt,
)
chain.invoke("sleep")

{'activity': 'sleep', 'text': 'Snooze'}

So, what the chain is doing is connecting these basic components (the LLM and the prompt template) into
a block that can be run separately. The chain allows you to turn workflows using LLLMs into this modular process of composing components.

Now, the newer versions of LangChain have a new representation language to create these chains (and more) known as LCEL or LangChain expression language, which is a declarative way to easily compose chains together. The same example as above expressed in this LCEL format would be:

In [15]:
chain = prompt | ChatOpenAI()

chain.invoke({"activity": "sleep"})

AIMessage(content='Snooze')

Notice that now the output is an `AIMessage()` object, which represents LangChain's way to abstract the output from an LLM model like ChatGPT or others.

These building blocks and abstractions that LangChain provides are what makes this library so unique, because it gives you the tools you didn't know you need it to build awesome stuff powered by LLMs.

# LangChain Exercise

Let's create a simple chain for summarization of content. 

Your chain should:

- A prompt template with one or more variables
- A model like ChatGPT or other (you can use local models if you'd like, I recommend `ChatOllama` for that!)
- Optional: use output parsing or just fetch the string output at the end!

## Example Answer

Let's make use of the `ChatPromptTemplate` to abstract away the following pieces of the prompt: 
- `content` - the text content to be summarized  
- `summary_format` - the format in which we want the summary to be presented (like bullet points and so on).

In [16]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Summarize this: {content}. The output should be in the following format: {summary_format}.")

# We can look at a simple example to illustrate what that prompt is doing
prompt.format(content="This is a test.", summary_format="One word summary")

'Human: Summarize this: This is a test.. The output should be in the following format: One word summary.'

Ok, now that we have our prompt template done, let's load the llm and create a nice chain to put everything together. 

In [17]:
from langchain_openai import ChatOpenAI

llm_chat =  ChatOpenAI()
chain = prompt | llm_chat # This is the Pipe symbol! from LCEL that connect model to prompt!

Now, that we have our chain we can run some tests. The cool thing about working with LLMs is that you can use them to create examples for simple tests like this (avoiding the annoynace of searching online for some piece of text, copying and pasting etc...). So, let's generate a few examples of tests below:

In [18]:
num_examples = 3
examples = []
for i in range(num_examples):
    examples.append(llm_chat.invoke("Create a piece of text with 2 paragraphs about a random topic regarding human-machine interaction."))

examples

[AIMessage(content="In today's society, human-machine interaction plays a crucial role in almost every aspect of our daily lives. From using smartphones to control smart home devices to relying on artificial intelligence for personalized recommendations, we are constantly engaging with machines to enhance our efficiency and convenience. However, this interaction is not without its challenges, as concerns about privacy, security, and ethical implications continue to arise.\n\nAs technology continues to advance, the line between humans and machines becomes increasingly blurred. With the development of robotics and artificial intelligence, there is a growing fear of machines surpassing human capabilities and potentially replacing us in various industries. This raises important questions about the future of work and the extent to which we are willing to integrate machines into our lives. Finding a balance between harnessing the power of technology and maintaining our humanity will be a key

Nice! Now that we have our examples, let's run our chain on them and check out the results.

In [19]:
summary_format = "bullet points"

outputs = []
for ex in examples:
    outputs.append(chain.invoke({"content": ex, "summary_format": summary_format}))

# Let's display one example output
outputs[0]

AIMessage(content="- Human-machine interaction is crucial in today's society, impacting daily activities such as controlling smart home devices and receiving personalized recommendations.\n- Challenges such as privacy, security, and ethical implications arise in human-machine interactions.\n- Technology advancements are blurring the line between humans and machines, raising concerns about machines surpassing human capabilities.\n- Questions about the future of work and integration of machines into our lives are important considerations in navigating human-machine interaction.\n- Finding a balance between technology and humanity will be a key challenge in the complexities of human-machine interaction in the future.")

Great! So it seems our chain worked and we generated some summaries! Let's visualize all the summaries generated in a neat way.

In [20]:
from IPython.display import Markdown

for i in range(num_examples):
    display(Markdown(f"Output {i} \n {outputs[i].content}"))
# Markdown(f"**Input**: {examples[0]}\n\n**Output**: {outputs[0]}")

Output 0 
 - Human-machine interaction is crucial in today's society, impacting daily activities such as controlling smart home devices and receiving personalized recommendations.
- Challenges such as privacy, security, and ethical implications arise in human-machine interactions.
- Technology advancements are blurring the line between humans and machines, raising concerns about machines surpassing human capabilities.
- Questions about the future of work and integration of machines into our lives are important considerations in navigating human-machine interaction.
- Finding a balance between technology and humanity will be a key challenge in the complexities of human-machine interaction in the future.

Output 1 
 - Human-machine interaction is prevalent in today's society, with technology playing a crucial role in daily lives
- Examples include voice assistants, self-driving cars, and smart home devices
- Interaction has made lives easier and more convenient, but has also raised questions about boundaries between humans and machines
- One challenge is ensuring machines can understand and respond to human emotions and intentions
- Machines struggle to interpret subtle cues like tone of voice or body language
- Researchers are working on developing more advanced AI algorithms to better understand human emotions
- Goal is to create machines that can perform tasks efficiently and interact with humans in a more natural and intuitive way

Output 2 
 - Human-machine interaction is increasingly important in our modern world
- Technology plays a major role in our daily lives, from smartphones to self-driving cars
- User interface design is a key aspect of human-machine interaction
- Well-designed interfaces can improve user experience, while poorly designed ones can lead to frustration
- Artificial intelligence and machine learning algorithms are important in human-machine interaction
- AI allows machines to learn from data and improve performance over time
- AI-powered systems can perform complex tasks like natural language processing and image recognition
- There are concerns about potential risks of AI, such as bias in algorithms and job loss due to automation
- Ethical implications should be considered in the design of machines to benefit society as a whole

Great! Our summaries worked, and we were able to apply a given summary format to all of them.

LangChain is an extremely powerful library to work with abstractions like these and throughout this course we hope to give you a gliimpse of the cool stuff you can build with it.